# CAICT 風機結冰預測 - Data Resampling

2017/07/12  
http://www.industrial-bigdata.com/competition/competitionAction!showDetail.action?competition.competitionId=1

<a id='home'></a>
## Outline
[Load Data and Modules](#load)  
[read raw data](#read)  
[assign event id](#assign)    
[get event summary](#eventsummary)  
[data resampling](#resampling)
[save resample data to csv](#csv)

<a id='load'></a>
**Load Data and Modules**

- **load Python modules:**

In [2]:
%matplotlib inline
import numpy as np
import datetime
import pandas as pd
import os
import sys
from PIL import Image
import lzma
import pickle
from IPython.display import display
import shutil
import csv
from sklearn.metrics import precision_score
from sklearn.metrics import roc_curve, auc  
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
from pylab import *
from IPython.core.interactiveshell import InteractiveShell
from operator import xor
#for resampling
#from imblearn.under_sampling import TomekLinks
#from imblearn.over_sampling import RandomOverSampler
#from imblearn.under_sampling import RandomUnderSampler
#from imblearn.over_sampling import SMOTE

import sklearn as sk
from sklearn.metrics import precision_score

pd.set_option('display.max_columns', None)
InteractiveShell.ast_node_interactivity = "all"


- ** data process functions **

In [3]:
def assign_eventid(data):
    label_col = data['label']
    label_col = label_col.values
    event_end=False
    event_id = 0;
    label_previous=0
    event_id_col = []
    for label in label_col: 
        if (label==0) & (label_previous==1):
            event_end=True
        label_previous = label;
        if event_end:
            event_end=False
            event_id=event_id+1
        event_id_col.append(event_id)    
    data['event']=event_id_col
    return data

def get_events_summary(data):
    events_info = pd.DataFrame(columns=('failStartTime','failEndTime','eventid', 'nor_min','unknown_min', 'abnor_min', 'nor_cnt', 'unknown_cnt', 'abnor_cnt','nor_idx_start','nor_idx_end','unknown_idx_start','abnor_idx_start', 'abnor_idx_end','no_unknown'))
    for eventid in set(data['event']):
        data_event = data[data['event']==eventid]
        evtdata_nor=data_event[data_event['label']==0]
        evtdata_abnor=data_event[data_event['label']==1]
        evtdata_unknown=data_event[data_event['label']==2]
    
        abnor_starttime = 0 if evtdata_abnor.empty else evtdata_abnor['time'].iloc[1]
        abnor_endtime = 0 if evtdata_abnor.empty else evtdata_abnor['time'].iloc[-1]
        
        nor_min = ((evtdata_nor['time'].iloc[-1] - evtdata_nor['time'].iloc[0]).total_seconds())/60
        abnor_min =  0 if evtdata_abnor.empty else ((evtdata_abnor['time'].iloc[-1] - evtdata_abnor['time'].iloc[0]).total_seconds())/60
        #unknown_min = ((evtdata_unknown['time'].iloc[-1] - evtdata_unknown['time'].iloc[0]).total_seconds())/60
        unknown_min = 0 if evtdata_unknown.empty else ((evtdata_unknown['time'].iloc[-1] - evtdata_unknown['time'].iloc[0]).total_seconds())/60    
        event_cnt = data_event['event'].count()
        nor_cnt = evtdata_nor['event'].count()
        abnor_cnt = 0 if evtdata_abnor.empty else evtdata_abnor['event'].count()
        #unknown_cnt= evtdata_unknown['event'].count()
        unknown_cnt = 0 if evtdata_unknown.empty else evtdata_unknown['event'].count()

        nor_idx_start= evtdata_nor.index[0]
        nor_idx_end= evtdata_nor.index[-1]
        
        abnor_idx_start= -1 if evtdata_abnor.empty else evtdata_abnor.index[0]
        abnor_idx_end= -1 if evtdata_abnor.empty else evtdata_abnor.index[-1]
        unknown_idx_start= abnor_idx_start if evtdata_unknown.empty else evtdata_unknown.index[0]

        no_known= 'Y' if evtdata_unknown.empty else 'N'
        events_info.loc[eventid] = [abnor_starttime, abnor_endtime, eventid, nor_min, unknown_min, abnor_min, nor_cnt, unknown_cnt, abnor_cnt,nor_idx_start,nor_idx_end, unknown_idx_start, abnor_idx_start, abnor_idx_end,no_known]
    return events_info


resample_policy_desc={
    '0': 'policy=0: no resample',
    '1': 'policy=1: event data + normal data (before and after event )',
    '2': 'policy=2: event data + normal data (before event only)'
}
%store resample_policy_desc

#根據事件的數據量, 從事件前後正常的區間取同樣多的數據
def resample_policy_p1(events_info):
    resample_info = pd.DataFrame(columns=('eventid', 'abnor_cnt','event_start', 'event_end'))
    for idx, row in events_info.iterrows():
        event_id = row['eventid']
        abnor_cnt = row['abnor_cnt']
        event_start = row['nor_idx_end'] - abnor_cnt
        event_end = row['abnor_idx_end'] + abnor_cnt
        if abnor_cnt==0:
            resample_info.loc[event_id] = [event_id, 0, 0, 0]
        else:
            resample_info.loc[event_id] = [event_id, abnor_cnt, event_start, event_end]
    return resample_info

#從不良事件的中間往前取pass:fail 2:1的數據
def resample_policy_p2(events_info):
    resample_info = pd.DataFrame(columns=('eventid', 'abnor_cnt','event_start', 'event_end'))
    for idx, row in events_info.iterrows():
        event_id = row['eventid']
        abnor_cnt = row['abnor_cnt']
        if abnor_cnt==0:
            resample_info.loc[event_id] = [event_id, 0, 0, 0]
            continue
        event_start = row['nor_idx_end']-abnor_cnt
        event_end = row['abnor_idx_start']+int(abnor_cnt/2)
        resample_info.loc[event_id] = [event_id, abnor_cnt, event_start, event_end]
    return resample_info

#根據事件的數據量, 從事件前正常的區間取同樣多的數據
def resample_policy_p3(events_info):
    resample_info = pd.DataFrame(columns=('eventid', 'abnor_cnt','event_start', 'event_end'))
    for idx, row in events_info.iterrows():
        event_id = row['eventid']
        abnor_cnt = row['abnor_cnt']
        event_start = row['nor_idx_end'] - abnor_cnt
        event_end = row['abnor_idx_end'] 
        if abnor_cnt==0:
            resample_info.loc[event_id] = [event_id, 0, 0, 0]
        else:
            resample_info.loc[event_id] = [event_id, abnor_cnt, event_start, event_end]
    return resample_info

#根據事件的數據量, 從事件前正常的區間取兩倍的數據
def resample_policy_p4(events_info):
    resample_info = pd.DataFrame(columns=('eventid', 'abnor_cnt','event_start', 'event_end'))
    for idx, row in events_info.iterrows():
        event_id = row['eventid']
        abnor_cnt = row['abnor_cnt']
        event_start = row['nor_idx_end'] - (2*abnor_cnt)
        event_end = row['abnor_idx_end'] 
        if abnor_cnt==0:
            resample_info.loc[event_id] = [event_id, 0, 0, 0]
        else:
            resample_info.loc[event_id] = [event_id, abnor_cnt, event_start, event_end]
    return resample_info


def get_resample_data(data, policyfun, evt_sumy):
    resample_info = policyfun(evt_sumy)
    resample_data = pd.DataFrame()
    for idx, row in resample_info.iterrows():
        event_id = row['eventid']
        abnor_cnt = row['abnor_cnt']
        event_start = row['event_start']
        event_end = row['event_end']
        resample_data_i = data[event_start:event_end].copy()
        resample_data_i['event']=event_id
        resample_data =resample_data.append(resample_data_i)
    return resample_data

Stored 'resample_policy_desc' (dict)



***

<a id='read'></a>
**read raw data** 

In [4]:
data_M15 = pd.read_csv('data/goodformat_15_FRESH.csv')
data_M21 = pd.read_csv('data/goodformat_21_FRESH.csv')



In [5]:
data_M15['label'].unique()

array([ 0.,  2.,  1.])

<a id='change'></a>
** change time format from string to datetime**

In [6]:
data_M15['time']=pd.to_datetime(data_M15['time'])
data_M21['time']=pd.to_datetime(data_M21['time'])

data_M15['timestamp'] = data_M15['time'].apply(lambda x: x.timestamp())
data_M21['timestamp'] = data_M21['time'].apply(lambda x: x.timestamp())


<a id='assign'></a>
** assign event id**

In [7]:
data_M15 = assign_eventid(data_M15)
data_M21 = assign_eventid(data_M21)

<a id='eventsummary'></a>
** get event summary **

In [8]:
events_summary_M15 = get_events_summary(data_M15)
events_summary_M21 = get_events_summary(data_M21)
events_summary_M15
events_summary_M21

,failStartTime,failEndTime,eventid,nor_min,unknown_min,abnor_min,nor_cnt,unknown_cnt,abnor_cnt,nor_idx_start,nor_idx_end,unknown_idx_start,abnor_idx_start,abnor_idx_end,no_unknown
0,2015-11-04 21:37:13,2015-11-04 22:29:33,0,4205.866667,1236.416667,52.450000,5727,860,443,0,5840,4756,6587,7029,N
1,2015-11-09 04:43:19,2015-11-09 06:35:39,1,5257.533333,118.400000,112.500000,43020,372,265,7030,50049,50050,50422,50686,N
2,2015-11-09 21:21:59,2015-11-09 23:14:41,2,413.383333,118.100000,112.816667,3383,1000,951,50687,54069,54070,55070,56020,N
3,2015-11-16 03:52:01,2015-11-16 16:08:25,3,8578.266667,637.350000,736.516667,39486,1114,2101,56021,95506,95507,95621,98721,N
4,2015-11-21 10:07:58,2015-11-21 10:20:07,4,4977.916667,16.416667,12.316667,35032,139,106,98722,133753,133754,133893,133998,N
5,2015-11-23 06:58:14,2015-11-23 07:39:43,5,2139.550000,27.300000,41.600000,14016,230,352,133999,148014,148015,148245,148596,N
6,2015-11-23 18:46:41,2015-11-26 01:40:43,6,296.983333,1478.766667,3294.200000,2120,2349,3188,148597,150716,150717,151717,156253,N
7,2015-11-29 02:04:48,2015-11-29 03:57:16,7,3799.033333,118.566667,112.583333,29550,1000,951,156254,185803,185804,186804,187754,N
8,2015-11-29 17:30:46,2015-11-30 06:50:45,8,153.416667,161.783333,800.100000,1321,1031,1254,187755,189075,189076,190107,191360,N
9,2015-11-30 23:22:15,2015-11-30 23:39:17,9,399.183333,0.000000,17.150000,3212,0,148,191361,194572,194573,194573,194720,Y


,failStartTime,failEndTime,eventid,nor_min,unknown_min,abnor_min,nor_cnt,unknown_cnt,abnor_cnt,nor_idx_start,nor_idx_end,unknown_idx_start,abnor_idx_start,abnor_idx_end,no_unknown
0,2015-11-04 22:15:45,2015-11-04 23:33:34,0,4448.650000,31.900000,77.933333,12020,270,659,0,12019,12020,12290,12948,N
1,2015-11-09 03:22:06,2015-11-09 05:14:21,1,5176.383333,118.400000,112.366667,40083,1000,747,12949,53031,53032,54032,54778,N
2,2015-11-09 21:26:19,2015-11-09 23:18:44,2,427.566667,118.233333,112.533333,2437,1000,951,54779,57215,57216,58216,59166,N
3,2015-11-13 02:59:13,2015-11-13 04:51:38,3,3595.716667,111.850000,112.533333,20717,575,951,59167,79883,79884,80459,81409,N
4,2015-11-16 15:31:48,2015-11-16 15:57:18,4,3325.250000,0.000000,25.616667,9716,0,217,81410,91125,91126,91126,91342,Y
5,2015-11-23 20:40:50,2015-11-23 22:33:13,5,9027.800000,118.500000,112.500000,48850,1000,434,91343,140192,140193,141193,141626,N
6,2015-11-24 05:42:08,2015-11-24 06:31:03,6,75.250000,93.516667,49.033333,637,791,414,141627,142263,142264,143055,143468,N
7,2015-11-24 14:59:02,2015-11-24 16:51:37,7,83.100000,118.550000,112.700000,477,1000,951,143469,143945,143946,144946,145896,N
8,2015-11-25 20:55:33,2015-11-26 03:40:39,8,55.900000,410.966667,405.216667,472,1222,1407,145897,146368,146369,147369,148997,N
9,2015-11-28 04:15:38,2015-11-28 06:08:06,9,2406.416667,118.383333,112.583333,17021,1000,951,148998,166018,166019,167019,167969,N


<a id='resampling'></a>
** data resampling **

- ** policy 1 **

In [9]:
resample_data_p1_M15 = get_resample_data(data_M15, resample_policy_p1, events_summary_M15)
resample_data_p1_M21 = get_resample_data(data_M21, resample_policy_p1, events_summary_M21)

- ** policy 2**

In [10]:
resample_data_p2_M15 = get_resample_data(data_M15, resample_policy_p2, events_summary_M15)
resample_data_p2_M21 = get_resample_data(data_M21, resample_policy_p2, events_summary_M21)

- ** policy 3**


In [11]:
resample_data_p3_M15 = get_resample_data(data_M15, resample_policy_p3, events_summary_M15)
resample_data_p3_M21 = get_resample_data(data_M21, resample_policy_p3, events_summary_M21)

- ** policy 4**

In [12]:
resample_data_p4_M15 = get_resample_data(data_M15, resample_policy_p4, events_summary_M15)
resample_data_p4_M21 = get_resample_data(data_M21, resample_policy_p4, events_summary_M21)

<a id='csv'></a>
** save resample data to csv**

In [14]:
resample_data_p0_M15 = data_M15

resample_data_p0_M15.to_csv('data/resample_data_p0_15.csv',index=False)
resample_data_p1_M15.to_csv('data/resample_data_p1_15.csv',index=False)
resample_data_p2_M15.to_csv('data/resample_data_p2_15.csv',index=False)
resample_data_p3_M15.to_csv('data/resample_data_p3_15.csv',index=False)
resample_data_p4_M15.to_csv('data/resample_data_p4_15.csv',index=False)

resample_data_p0_M21 = data_M21
resample_data_p0_M21.to_csv('data/resample_data_p0_21.csv',index=False)
resample_data_p1_M21.to_csv('data/resample_data_p1_21.csv',index=False)
resample_data_p2_M21.to_csv('data/resample_data_p2_21.csv',index=False)
resample_data_p3_M21.to_csv('data/resample_data_p3_21.csv',index=False)
resample_data_p4_M21.to_csv('data/resample_data_p4_21.csv',index=False)


In [15]:
policy0= 'no resample'
policy1= 'event data + normal data (before and after event )'
policy2= 'half event data + normal data (before event only)  '
policy3= 'event data + normal data (before event only) '
policy4= 'event data + 2*normal data (before event only) '
policy_desc = [policy0,policy1,policy2,policy3,policy4]
policy_df = pd.DataFrame({'policy_id':range(len(policy_desc)), 'policy_desc':policy_desc})
policy_df = policy_df.set_index('policy_id')
policy_df.to_csv('data/resample_policy_desc.csv',index=False)

[home](#home)

***